In [113]:
import copy;
import json;
import requests;
from bs4 import BeautifulSoup;

def getSoup(inurl):
    r = requests.get(inurl)
    r.encoding = 'utf-8'
    soup=BeautifulSoup(r.text);
    return soup;
#end of getSoup

def gen_page_number_list():
    result=list();
    magical_number=21;#this is found by trying manually
    for i in range(1,magical_number+1):
        sup=getSoup("https://movies.yahoo.com.tw/moviegenre_result.html?genre_id="+str(i)+"&page=1");
        page_count=1;
        if sup.find("div",class_="page_numbox") is not None:
            lis=sup.find("div",class_="page_numbox").find_all("li");
            page_count=int(lis[len(lis)-3].text);
        #print({"id":i,"page_count":page_count});
        result.append({"genre_id":i,"page_count":page_count});
    return result;
#end of gen_page_number_list

def extend_page_dataset(indataset,inurl):
    if inurl in indataset["links"]:
        return indataset;
    
    a_data={
        "name":"",
        "link":"",
        "when":"",
        "desc":"",
        "class":[]
    };
    sup=getSoup(inurl);
    info_part=sup.find("div",class_="movie_intro_info_r");
    desc_str=sup.find("span",id="story").text.strip();
    
    when_str="";
    for entry in info_part.find_all("span"):
        if entry.text.startswith("上映日期："):
            when_str=copy.deepcopy(entry.text);
            when_str=when_str[len("上映日期："):];
            break;
    if when_str=="":
        raise Exception("date not found");
        
    class_list=[];
    for entry in info_part.find("div",class_="level_name_box").find_all("a"):
        class_list.append(entry.text.strip());
    
    
    a_data["name"]=info_part.find("h1").text;
    a_data["link"]=inurl;
    a_data["when"]=when_str;
    a_data["desc"]=desc_str;
    a_data["class"]=class_list;
    
    indataset["data"].append(a_data);
    indataset["links"].append(inurl);
    return indataset;
#end of extend_page_dataset


def get_link_of_pages(inurl):
    result=[];
    sup=getSoup(inurl);
    lis=sup.find("ul",class_="release_list").find_all("li");
    for entry in lis:
        url=entry.find("a",class_="gabtn")["href"];
        result.append(url);
    return result;
#end of get_link_of_pages

def dataset_update(dataset={"data":[],"links":[]}):
    for entry in gen_page_number_list()[0:1]:
        for pc in range(1,entry["page_count"]-117):
            url="https://movies.yahoo.com.tw/moviegenre_result.html?"
            url+="genre_id="+str(entry["genre_id"]);
            url+="&page="+str(pc);
            for info_url in get_link_of_pages(url):
                extend_page_dataset(dataset,info_url);
            print(url);
            if len(dataset["links"])>10:
                break;
        if len(dataset["links"])>10:
                break;
    return dataset;
#end of dataset_update

def homework_2():
    things=dataset_update();
    with open("./output.txt","x",encoding='utf-8') as ifile:
        ifile.write(json.dumps(things));
    print("done!");
    with open("./output.txt",encoding='utf-8') as ifile:
        print(json.loads(ifile.read()));
    print("done!");
#end of homework_2


homework_2();

https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=1
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=2
done!
{'data': [{'name': '小小兵2：格魯的崛起', 'link': 'https://movies.yahoo.com.tw/movieinfo_main/%E5%B0%8F%E5%B0%8F%E5%85%B52-%E6%A0%BC%E9%AD%AF%E7%9A%84%E5%B4%9B%E8%B5%B7-minions-the-rise-of-guru-10493', 'when': '2022-07-01', 'desc': '今年暑假，影史最賣座的動畫電影系列之一和風靡全球的現象，即將帶來關於一個12歲小男孩的故事，他夢想成為世上最偉大的超級大壞蛋。這部全新動畫喜劇動作片就是《小小兵2：格魯的崛起》。\r\n\xa0\r\n這部電影充滿了照明娛樂招牌的顛覆性和無厘頭喜劇橋段、各種大眾文化的笑梗、感人肺腑的劇情、勁爆的電影配樂，以及宇宙超級無敵誇張的動作場面。《小小兵2：格魯的崛起》一片由這個電影系列的原創掌舵，製片包括照明娛樂極具遠見的創辦人和執行長克里斯梅勒丹德利、他的長期合作夥伴珍娜海莉，以及克里斯雷納德，導演凱爾巴爾達（《神偷奶爸3》、《小小兵》）則再度回歸執導這部電影。共同導演是布萊德艾柏森（長壽喜劇影集《辛普森家庭》）以及強納森戴凡（《寵物當家》系列電影）。', 'class': ['動畫', '動作', '喜劇']}, {'name': '蝙蝠俠', 'link': 'https://movies.yahoo.com.tw/movieinfo_main/%E8%9D%99%E8%9D%A0%E4%BF%A0-the-batman-10904', 'when': '2022-03-04', 'desc': '《蝙蝠俠》將聚焦於一個年輕版本的布魯斯韋恩，並從中檢視他在面罩之下的靈魂。羅伯派汀森將在新版《蝙蝠俠》中飾演這位DC超級英雄，電影是由《猩球崛起：終極決戰》導演麥特李維斯（Matt Reeves）所執導。', 'class': ['動作', '犯罪']